In [ ]:
%load_ext autoreload
%autoreload 2
from astropy.coordinates import Distance
import bilby
from gwBackground import *
import matplotlib.pyplot as plt
import numpy as np
import os
from pygwb.constants import H0
H0_si = H0.si.value
from pygwb.detector import Interferometer
from pygwb.network import Network
from pygwb.parameters import Parameters
from pygwb.baseline import Baseline
from pygwb.simulator import Simulator
import scipy
from scipy.integrate import trapz
from scipy.integrate import cumtrapz
from scipy.special import erfinv
import sys
sys.path.append('../code/')
from tqdm import tqdm

In [ ]:
import matplotlib 
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
fontsize = 9
# ALWAYS USE figsize = (3.375, X) for column plots 
# figsize = (6.75, X) for rows 
params = {
   'axes.labelsize': fontsize,
   'font.size': fontsize,
   'legend.fontsize': 8,
   'xtick.labelsize': fontsize,
   'ytick.labelsize': fontsize,
   'axes.titlesize':fontsize,
   'lines.linewidth':1,  
   'xtick.direction':'in',
   'ytick.direction':'in',
#    'text.usetex': True,
   'font.family':'Serif',
   'font.serif':'Hoefler Text',
   'axes.grid':True,
   'figure.figsize': (6.75, 4),
   'figure.dpi':250,
   'mathtext.fontset':'cm'
}

for param in params.keys():
    matplotlib.rcParams[param] = params[param]    

In [ ]:
# Calculate the merger rate
def R(alpha, beta, z, z_p, R0):
    '''
    Equation from Callister et al. 2020.
    Calculates the merger rate at a given redshift.
    
    Parameters
    ----------
    alpha : double
        spectral index before peak z_p
    beta : double
        spectral index after peak z_p
    z : double
        redshift
    z_p : double
        peak redshift of merger rate
    R0 : double
        current merger rate (z = 0)
        
    Returns
    -------
    double
        merger rate at the given redshift
    '''
    return (C(alpha, beta, z_p)*((R0*((1+z)**alpha))/(1+(((1+z)/(1+z_p))**(alpha+beta)))))

def C(alpha, beta, z_p):
    '''
    Equation from Callister et al. 2020.
    Calculates the normalization constant for the merger rate.
    
    Parameters
    ----------
    alpha : double
        spectral index before peak z_p
    beta : double
        spectral index after peak z_p
    z_p : double
        peak redshift of merger rate
    
    Returns
    -------
    double
        normalization constant for the merger rate
    '''
    return (1+((1+z_p)**(-alpha-beta)))

def Hubble_rate(z, H0, omega_R, omega_M, omega_k, omega_lambda):
    '''
    Equation from Renzini et al. 2022.
    Calculates the Hubble rate as a function of redshift.
    
    Parameters
    ----------
    z : double
        redshift
    H0 : double
        Hubble constant (Hubble rate at z = 0)
    omega_R : double
        radiation component of energy density
    omega_M : double
        matter component of energy density
    omega_k : double
        spacetime curvature component of energy density
    omega_lambda : double
        dark energy component of energy density, cosmological constant
    
    Returns
    -------
    double
        Hubble rate for a given redshift z
    '''
    return H0*(((omega_R*((1+z)**4))+(omega_M*((1+z)**3))+(omega_k*((1+z)**2))+omega_lambda)**(1/2))

alpha = 1.9
beta = 3.4
z_p = 2.4
z_max = 10

R0 = 28.3 # current merger rate

dz = 0.01
zsMerger = np.arange(0.,z_max,dz)

mergerRate = []
for i in range(len(zsMerger)):
    mergerRate.append(R(alpha, beta, zsMerger[i], z_p, R0))
mergerRate = np.array(mergerRate)

In [ ]:
# Define redshift prior
from bilby.gw.prior import Cosmological
class PowerLawRedshiftPrior(Cosmological):
    def __init__(self, minimum, maximum, R0, alpha, beta, zp, **kwargs):
        self.R0 = R0
        self.alpha = alpha
        self.beta = beta
        self.zp = zp
        super(PowerLawRedshiftPrior, self).__init__(minimum=minimum, maximum=maximum, **kwargs)
        
    def _get_redshift_arrays(self):
        zs = np.linspace(self._minimum['redshift'] * 0.99,
                         self._maximum['redshift'] * 1.01, 1000)
        C = 1 + (1 + self.zp)**(-self.alpha - self.beta)
        p_dz = (1/(1+zs)) * C * ((self.R0 * (1 + zs)**(self.alpha))/(1 + ((1 + zs)/(1 + self.zp))**(self.alpha + self.beta))) * 4 * np.pi * self.cosmology.differential_comoving_volume(zs).value
        
        return zs, p_dz

In [ ]:
fig, ax = plt.subplots()
ax.set_title('$\Omega_{GW}(f)$ vs. Frequency')
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel(r'$\Omega_{GW}(f)$')
ax.set_xlim(10, 500)

omega_f_TCs = []
max_freqs_TC = []
max_omega_f_TC = []

max_mass_range = [38.9, 41.45, 44, 48.6, 53.2]
for max_mass in max_mass_range:
    m1_min = 5.                         # Minimum BH mass
    m1_max = max_mass                   # Maximum BH mass
    m2_min = 5.
    m2_max = max_mass
    fmax = 3000.                        # Maximum frequency we want to consider
    inspiralOnly=False                  # Include contributions from merger and ringdown
    minimum_component_mass = m2_min
    maximum_component_mass = m1_max
    
    # Define priors for the parameters in the Regimbau method
    priors = bilby.gw.prior.BBHPriorDict(aligned_spin=True)
    # priors['chirp_mass'] = bilby.core.prior.Uniform(2, 30, name="chirp_mass")
    # priors['mass_ratio'] = 1.0
    # priors['mass_1'] = bilby.core.prior.LogUniform(5,m1_max)
    # priors['mass_2'] = bilby.core.prior.Uniform(5,m2_max)
    priors['mass_1'] = bilby.core.prior.PowerLaw(alpha=-2.3, minimum=5, maximum=max_mass)
    priors['mass_ratio'] = bilby.core.prior.PowerLaw(alpha=1.5, minimum=0, maximum=1)
    priors['chi_1'] = 0
    priors['chi_2'] = 0
    priors['theta_jn'] = bilby.core.prior.Uniform(0, 2*np.pi, name='theta_jn') # bilby.core.prior.Cosine()
    priors['redshift'] = PowerLawRedshiftPrior(R0=R0, alpha=alpha, beta=beta, zp=z_p, name='redshift', minimum=0, maximum=10)

    print('Defining OmegaGW_BBH for max_mass = ' + str(max_mass))
    omg = OmegaGW_BBH(minimum_component_mass,maximum_component_mass,zsMerger)

    # Calculate probabilities based on the Regimbau method
    print('Calculating probability grid.')
    # Priors defined in (m1, m2) space
#     probs = np.empty((omg.m1s_2d.shape[0],omg.m2s_2d.shape[1])) # initialize array
#     for i in range(omg.m1s_2d.shape[0]): # for each m1
#         for j in range(omg.m2s_2d.shape[1]): # for each m2
#             prob = priors.prob({'mass_1':omg.m1s_2d[i][j], 'mass_2':omg.m2s_2d[i][j]}) # calculate probability
#             probs[i][j] = prob # insert probability into probability array
#     probs = probs*(omg.Mtots_2d**2./(1.+omg.qs_2d)**2.) # multiply by the Jacobian
     
    # Priors defined in (m1, q) space
    probs = np.empty((omg.m1s_2d.shape[0],omg.qs_2d.shape[1])) # initialize array
    for i in range(omg.m1s_2d.shape[0]): # for each m1
        for j in range(omg.qs_2d.shape[1]): # for each m2
            prob = priors.prob({'mass_1':omg.m1s_2d[i][j], 'mass_ratio':omg.qs_2d[i][j]}) # calculate probability
            probs[i][j] = prob # insert probability into probability array
    probs = probs*(omg.Mtots_2d/(1.+omg.qs_2d)) # multiply by the Jacobian

    # Now make sure that we're specifying physical masses for our BH and NS components
    probs[omg.m1s_2d<m1_min] = 0
    probs[omg.m1s_2d>m1_max] = 0
    probs[omg.m2s_2d<m2_min] = 0
    probs[omg.m2s_2d>m2_max] = 0

    # Finally, normalize the probabilities and give them to our OmegaGW calculator!
    probs /= np.sum(probs)
    omg.probs = probs

    freqs_TC = np.logspace(0,3.2,200)
    print('Evalating max_mass = ' + str(max_mass))
    omega_f = omg.eval(R0,mergerRate,freqs_TC)
    print('Evaluated max_mass = ' + str(max_mass))
    omega_f_TCs.append(omega_f)
    
    print('Finding maximum point.')
    max_omega_f_TC.append(max(omega_f))
    for i in range(len(omega_f)):
        if omega_f[i] == max(omega_f):
            max_freqs_TC.append(freqs_TC[i])
            
    if max_mass == max_mass_range[0]: 
        ax.plot(freqs_TC, omega_f, color='#a1d6ff', label=r'$m_{max}=$'+str(max_mass)+r' $M_\odot$')
    if max_mass == max_mass_range[1]: 
        ax.plot(freqs_TC, omega_f, color='#5db0f0', label=r'$m_{max}=$'+str(max_mass)+r' $M_\odot$')
    if max_mass == max_mass_range[2]: 
        ax.plot(freqs_TC, omega_f, color='#2f82c2', label=r'$m_{max}=$'+str(max_mass)+r' $M_\odot$')
    if max_mass == max_mass_range[3]: 
        ax.plot(freqs_TC, omega_f, color='#115385', label=r'$m_{max}=$'+str(max_mass)+r' $M_\odot$')
    if max_mass == max_mass_range[4]: 
        ax.plot(freqs_TC, omega_f, color='#052945', label=r'$m_{max}=$'+str(max_mass)+r' $M_\odot$')

ax.legend()
fig.show()

In [ ]:
fig, ax = plt.subplots()
ax.set_title('$\Omega_{GW}(f)$ vs. Frequency')
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel(r'$\Omega_{GW}(f)$')
ax.plot(freqs_TC, omega_f_TCs[0], color='#a1d6ff', label=r'$m_{2, max}=$'+str(max_mass_range[0])+r' $M_\odot$')
ax.plot(freqs_TC, omega_f_TCs[1], color='#5db0f0', label=r'$m_{2, max}=$'+str(max_mass_range[1])+r' $M_\odot$')
ax.plot(freqs_TC, omega_f_TCs[2], color='#2f82c2', label=r'$m_{2, max}=$'+str(max_mass_range[2])+r' $M_\odot$')
ax.plot(freqs_TC, omega_f_TCs[3], color='#115385', label=r'$m_{2, max}=$'+str(max_mass_range[3])+r' $M_\odot$')
ax.plot(freqs_TC, omega_f_TCs[4], color='#052945', label=r'$m_{2, max}=$'+str(max_mass_range[4])+r' $M_\odot$')
ax.legend()

In [ ]:
fig, ax = plt.subplots()
ax.set_title(r'$\Omega_{GW, max}$ vs. $m_{2, max}$')
ax.set_xlabel(r'$m_{2, max}$')
ax.set_ylabel(r'$\Omega_{GW, max}$')
ax.scatter(max_mass_range, max_omega_f_TC, color='#000000')
a, b = np.polyfit(max_mass_range, max_omega_f_TC, 1)
omegaGW_max = []
for i in range(len(max_mass_range)):
    omegaGW_max.append(a*max_mass_range[i]+b)
ax.plot(max_mass_range, omegaGW_max, color='#000000')

fig.show()

In [ ]:
fig, ax = plt.subplots()
ax.set_title(r'$f_{max}$ vs. $m_{2, max}$')
ax.set_xlabel(r'$m_{2, max}$')
ax.set_ylabel(r'$f_{max}$')
ax.scatter(max_mass_range, max_freqs_TC, color='#000000')
a, b = np.polyfit(max_mass_range, max_freqs_TC, 1)
f_max = []
for i in range(len(max_mass_range)):
    f_max.append(a*max_mass_range[i]+b)
ax.plot(max_mass_range, f_max, color='#000000')
# ax.text(r'$f_{max}$ = ' + '{:.2f}'.format(a) + r'$m_{2,max}$' + ' + {:.2f}'.format(b))
fig.show()